In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image # dataset label is not used. replaced with the same image input.

# parameters
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024


### START CODE HERE (Replace instances of `None` with your code) ###

# use tfds.load() to fetch the 'train' split of CIFAR-10
train_dataset = tfds.load('cifar10', split='train', as_supervised=True)

# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image)

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


# use tfds.load() to fetch the 'test' split of CIFAR-10
test_dataset = tfds.load('cifar10', split='test', as_supervised=True)

# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image)

# batch the dataset
test_dataset = test_dataset.batch(BATCH_SIZE)

### END CODE HERE ###

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete9CT2UF/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete9CT2UF/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [5]:
# suggested layers to use. feel free to add or remove as you see fit.
from keras.layers import Conv2D, UpSampling2D
from keras import backend as K

K.clear_session()

# use the Sequential API (you can remove if you want to use the Functional API)
model = Sequential()

### START CODE HERE ###
# use `model.add()` to add layers (if using the Sequential API)
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="relu", input_shape=(32, 32, 3))) # (16, 16, 64)
model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="relu")) # (8, 8, 128)

model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu")) # (8, 8, 256)

model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu")) # (16, 16, 128)
model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu")) # (32, 32, 64)
model.add(UpSampling2D(size=(2, 2)))

model.add(Conv2D(filters=3, kernel_size=(3, 3), padding="same", activation="sigmoid")) # (32, 32, 1)


# Please do not change the model.compile() parameters
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

### END CODE HERE ###

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 16, 16, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 128)         73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 256)         295168    
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         295040    
                                                                 
 up_sampling2d (UpSampling2D  (None, 16, 16, 128)      0         
 )                                                               
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 16, 64)        73792     
                                                        

In [8]:
# parameters (feel free to change this)
train_steps = len(train_dataset) // BATCH_SIZE
val_steps = len(test_dataset) // BATCH_SIZE

### START CODE HERE ###
model.fit(train_dataset,
          steps_per_epoch=train_steps,
          validation_data=test_dataset,
          validation_steps=val_steps,
          epochs=100)
### END CODE HERE ###

Epoch 1/100
3/3 [==============================] - 0s 55ms/step - loss: 0.0076 - accuracy: 0.7691
Epoch 2/100
3/3 [==============================] - 0s 45ms/step - loss: 0.0067 - accuracy: 0.7527
Epoch 3/100
3/3 [==============================] - 0s 51ms/step - loss: 0.0061 - accuracy: 0.7685
Epoch 4/100
3/3 [==============================] - 0s 43ms/step - loss: 0.0061 - accuracy: 0.7483
Epoch 5/100
3/3 [==============================] - 0s 45ms/step - loss: 0.0059 - accuracy: 0.7639
Epoch 6/100
3/3 [==============================] - 0s 41ms/step - loss: 0.0055 - accuracy: 0.7570
Epoch 7/100
3/3 [==============================] - 0s 41ms/step - loss: 0.0058 - accuracy: 0.7544
Epoch 8/100
3/3 [==============================] - 0s 48ms/step - loss: 0.0054 - accuracy: 0.7551
Epoch 9/100
3/3 [==============================] - 0s 41ms/step - loss: 0.0054 - accuracy: 0.7624
Epoch 10/100
3/3 [==============================] - 0s 44ms/step - loss: 0.0050 - accuracy: 0.7523
Epoch 11/100
3/3 [=

In [9]:
result = model.evaluate(test_dataset, steps=10)

10/10 [==============================] - 0s 21ms/step - loss: 0.0035 - accuracy: 0.7963
